In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Get data from WHO: https://www.who.int/data/gho/data/indicators/indicator-details/GHO/gho-tobacco-control-monitor-current-tobaccouse-tobaccosmoking-cigarrettesmoking-agestd-tobagestdcurr
tobacco_use_full = pd.read_csv('./TobaccoUseData.csv')
tobacco_use_full.head(1)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,M_Est_tob_curr_std,Estimate of current tobacco use prevalence (%)...,numeric,SEAR,South-East Asia,Country,PRK,Democratic People's Republic of Korea,Year,2025,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Projected from surveys completed prior to 2020...,EN,2022-01-16T23:00:00.000Z


In [8]:
tobacco_use = tobacco_use_full[['Period', 'ParentLocation', 'Location type', 'Location', 'Value']]
tobacco_use.head(3)

,Period,ParentLocation,Location type,Location,Value
0,2025,South-East Asia,Country,Democratic People's Republic of Korea,0.0
1,2025,Europe,Country,Azerbaijan,0.1
2,2025,Africa,Country,Eritrea,0.2


In [9]:
per_continent_tobacco_use = tobacco_use[tobacco_use['Period'] == 2022]

SyntaxError: invalid syntax (2038700554.py, line 1)